<!-- UPDATE: pin to particular commit -->
<a href="https://colab.research.google.com/github/imagej/pyimagej/blob/main/doc/llms/pyimagej-ai-guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔬 Your AI Guide to PyImageJ

Welcome! This notebook serves as your personal [PyImageJ](https://pyimagej.readthedocs.io/) learning environment. Here we'll set up PyImageJ for scientific image analysis and configure the Gemini AI for support.

Combining ImageJ, Python, LLMs, and Notebooks is a lot! If any of this is new to you, start with this notebook's [companion documentation](https://pyimagej.readthedocs.io/AI-Tutorial-Notebook.html).

## 📝 Notes on Google Colab
Colab is fundamentally a *non-reproducible environment*:
- We do not have control over what is installed in the base environment, let alone the Python version itself
- ❗Therefore we **strongly recommend against** publishing workflows that must run in Colab❗

⚠️ **Note:** If you see `This notebook was not authored by Google` or `Could not access the resources needed to display output` warnings, you can safely ignore them

## 🧭 How to Use This Notebook

**Step 1: Run Everything!** 
- ⚠️ Setting up this notebook can take **a few minutes**
- First, make your own copy of the notebook with `File > Save a copy in Drive`
- Then use the `Run all` toolbar button above (`Ctrl+F9`) to start the setup process

In [ ]:
#@title ⌛ Setup Status { display-mode: "form" }

from IPython.display import display
import ipywidgets as widgets

# Create a progress widget
progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    orientation='horizontal',
    layout=widgets.Layout(width='60%', margin='0 10px 0 0')

)
status_label = widgets.Label(value='Initializing...', layout=widgets.Layout(width='40%'))
display(widgets.HBox(
    [status_label, progress],
    layout=widgets.Layout(align_items='center')
))

**Step 2: Start Exploring with Gemini**
- If everything looks good, proceed to [🚀 Start Your ImageJ Journey](#scrollTo=_Start_Your_ImageJ_Journey) and start a conversation with Gemini about what to do next!

⚠️ **Note**: If anything went wrong during setup, head to [🛠️ Troubleshooting](#scrollTo=_Troubleshooting)

# ⚙️ Notebook Configuration

In [ ]:
%%capture clone_output
#@title 📦 Download LLM Rules { display-mode: "form" }
#@markdown This notebook serves as a modular framework for learning with Large Language Models (LLMs).
#@markdown
#@markdown To "educate" your LLM, customizing how Gemini responds to your prompts, we need to download
#@markdown the PyImageJ repository, which contains instructional files loaded in [🤖 Customize Gemini](#scrollTo=_Customize_Gemini).

# Clone PyImageJ repository for LLM instructional files
import os

status_label.value = 'Downloading AI files...'

if not os.path.exists("pyimagej"):
    print("⬇️ Cloning PyImageJ repository for templates and examples...")
    !git clone https://github.com/imagej/pyimagej.git
    # --UPDATE: pin to particular commit--
    #!cd /content/pyimagej && git checkout 27d5781849d6764b575a0a9b67e7e0766f263c8e
    print("✅ PyImageJ repository cloned successfully!")
else:
    print("✅ PyImageJ repository already available")

In [ ]:
#@title 🤖 Customize Gemini { display-mode: "form", run: "auto" }
#@markdown ⚠️The output of this cell is **just for Gemini** - not fit for human consumption!⚠️
#@markdown 
#@markdown This cell tailors Gemini's persona and provides it with fundamental information for using PyImageJ.
#@markdown
#@markdown We take advantage of Gemini's "knowledge" of all cell outputs in the notebook: all we need to do is print supporting text and Gemini will automatically pick it up.

progress.value += 5
status_label.value = 'Loading AI files...'

# Activity file mapping for different experience categories
from pathlib import Path
personas_dir = Path('personas')
activity_dir = personas_dir / 'activities'
rules_dir = Path('rulesets')
environment_dir = rules_dir / 'environments'

def load_llm_file(filename):
	"""Load a template file with clear section markers"""
	try:
		# First check if we're in a cloned pyimagej repo
		file_path = Path('/content/pyimagej/doc/llms') / filename
		if not file_path.exists():
			# Fall back to current directory structure
			file_path = Path('./pyimagej/doc/llms') / filename
			
		if file_path.exists():
			content = file_path.read_text(encoding='utf-8')
			# Add clear section markers for LLM parsing
			base_name = Path(filename).stem  # Remove directory and .md extension
			section_name = base_name.replace('_', ' ').upper()
			return f"\n===== START OF {section_name} =====\n{content}\n===== END OF {section_name} =====\n"
		else:
			return f"# File not found: {filename}\n"
	except Exception as e:
		return f"# Error loading {filename}: {e}\n"

# Load base persona with section markers
gemini_text = load_llm_file(personas_dir / 'base_persona.md')
gemini_text += load_llm_file(rules_dir / 'pyimagej_core.md')
gemini_text += load_llm_file(rules_dir / 'imagej1_api.md')
gemini_text += load_llm_file(rules_dir / 'imagej_ops.md')
gemini_text += load_llm_file(environment_dir / 'env_colab.md')

# Register the custom persona and rules with the LLM
print(gemini_text)

In [ ]:
%%capture bundle_setup
#@title 🏝️ Setup Fiji Bundle { display-mode: "form" }
#@markdown PyImageJ connects ImageJ to the Python ecosystem, but we still need an *ImageJ application* with all our image analysis goodness.
#@markdown
#@markdown In this cell we download a "local" copy of the selected Fiji bundle, which we access via PyImageJ. This lets us use all of the plugins you get when using Fiji normally. Note that this can take **a few minutes**.
#@markdown
#@markdown 💡 Tip: this setup will initialize the **`ij` variable** -  your main interface to ImageJ (e.g., `ij.io().open()`, `ij.op().math()`)

# --UPDATE: pin to particular release--
bundle_id= "20251023"

import os
import subprocess
import time
import re
from typing import Any

# Set target bundle (update as desired from https://github.com/fiji/fiji-builds/releases)
bundle_name = "fiji-py-colab-" + bundle_id

progress.value = 10
status_label.value = 'Preparing to download Fiji...'

# Only process bundle if Fiji doesn't already exist
if not os.path.exists("Fiji"):
    print("Setting up Fiji Colab bundle...")

    # Work in a safe bundle directory to avoid conflicts with existing caches
    bundle_dir = "fiji_colab_bundle"
    original_dir = os.getcwd()

    # Create and enter bundle directory
    os.makedirs(bundle_dir, exist_ok=True)
    os.chdir(bundle_dir)

    # Download bundle
    file_name = "{bundle_name}.tar.gz".format(bundle_name=bundle_name)
    url = "https://github.com/fiji/fiji-builds/releases/download/{bundle_name}/{bundle_name}.tar.gz".format(bundle_name=bundle_name)
    print("Downloading Fiji Colab bundle from {url} ...".format(url=url))
    
    # Keep trying until the file is complete
    max_retries = 3
    retry_count = 0
    while retry_count < max_retries:
        proc = subprocess.Popen(
            ["wget", "--progress=bar:force", "-c", url, "-O", file_name],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
            bufsize=1
        )
        
        # Monitor wget progress in real-time
        status_label.value = 'Downloading Fiji bundle (~300MB)...'
        for line in proc.stderr:
            # wget outputs progress to stderr like: "12% [=====>     ] 1,234,567   123KB/s"
            match = re.search(r'(\d+)%', line)
            if match:
                wget_percent = int(match.group(1))
                # Map wget 0-100% to notebook 20-70%
                progress.value = 10 + int(wget_percent * 0.4)
                status_label.value = f'Downloading Fiji bundle...'
        
        proc.wait()
        
        if proc.returncode == 0:
            print("Download finished successfully.")
            break
        elif proc.returncode == 8:  # wget returns 8 for 404/server errors
            stderr_output = proc.stderr if isinstance(proc.stderr, str) else "Unknown error"
            print(f"❌ Download failed: {stderr_output}")
            raise FileNotFoundError(
                f"Bundle not found at {url}. "
                f"Please check that bundle_id '{bundle_id}' exists at "
                f"https://github.com/fiji/fiji-builds/releases"
            )
        else:
            retry_count += 1
            if retry_count < max_retries:
                print("Download interrupted (return code: {}). Retrying in 10 seconds... ({}/{})".format(
                    proc.returncode, retry_count, max_retries))
                status_label.value = f'Retrying download ({retry_count}/{max_retries})...'
                time.sleep(10)
            else:
                print(f"❌ Download failed after {max_retries} attempts")
                raise RuntimeError(f"Failed to download bundle after {max_retries} attempts. Last error code: {proc.returncode}")

    # Extract bundle
    progress.value = 50
    status_label.value = 'Extracting bundle...'
    print("Extracting bundle...")
    !tar -xzf {file_name}

    # Move Fiji and Java to working directory
    progress.value = 60
    status_label.value = 'Moving files...'
    !mv Fiji ../
    !mv jdk-latest ../
    print("Moved Fiji and Java runtime to working directory")

    # Safely merge bundle caches with existing user caches
    progress.value = 65
    status_label.value = 'Merging caches...'
    print("Merging bundle caches with local caches...")

    # Merge .jgo cache
    !mkdir -p ~/.jgo
    !cp -r .jgo/* ~/.jgo/ || true
    print("Merged .jgo cache")

    # Merge .m2 cache
    !mkdir -p ~/.m2
    !cp -r .m2/* ~/.m2/ || true
    print("Merged .m2 cache")

    # Clean up bundle directory and return to original location
    progress.value = 70
    status_label.value = 'Cleaning up...'
    os.chdir(original_dir)
    !rm -rf {bundle_dir}
    print("Bundle setup complete!")
else:
    print("Fiji already exists, skipping bundle setup")

# Set up Java environment
progress.value = 72.5
status_label.value = 'Configuring Java environment...'
if "JAVA_HOME" not in os.environ:
    java_home = !find ./jdk-latest/linux64 -name "*jdk*" -type d
    os.environ['JAVA_HOME'] = java_home[0]
    print(f"Set JAVA_HOME to: {os.environ['JAVA_HOME']}")

# Install system dependencies only if needed
progress.value = 75
status_label.value = 'Installing system dependencies...'
!command -v mvn >/dev/null 2>&1 && command -v xdpyinfo >/dev/null 2>&1 || (apt-get update && apt-get install -y maven xvfb x11-utils libxext6 libxrender1 libxtst6 libxi6 fonts-dejavu)

# Start up a virtual display only if needed
progress.value = 80
status_label.value = 'Setting up virtual display...'
if "DISPLAY" not in os.environ:
    print("Setting up virtual display...")
    os.environ["DISPLAY"] = ":99"
    xvfb_process = subprocess.Popen(["Xvfb", ":99", "-screen", "0", "1024x768x24"])

    # Wait a moment for Xvfb to start
    time.sleep(2)

    # Test if display is working
    try:
        result = subprocess.run(["xdpyinfo"], capture_output=True, text=True, timeout=5)
        display_working = result.returncode == 0
        if display_working:
            print("✓ Virtual display working - using interactive mode")
            ui_mode = 'interactive'
        else:
            print("⚠ Virtual display failed - falling back to headless mode")
            ui_mode = 'headless'
    except (subprocess.TimeoutExpired, FileNotFoundError):
        print("⚠ Cannot test display - falling back to headless mode")
        ui_mode = 'headless'
else:
    print(f"Display already configured: {os.environ['DISPLAY']}")

# Install PyImageJ if not already installed
progress.value = 85
status_label.value = 'Installing PyImageJ...'
try:
    import imagej
    print("PyImageJ already installed")
except ImportError:
    print("Installing PyImageJ...")
    %pip install pyimagej
    import imagej

# This line avoids linting warnings about being unbounded vars
ij: Any = ij if 'ij' in globals() or 'ij' in locals() else None
ui_mode: Any = ui_mode if 'ui_mode' in globals() or 'ui_mode' in locals() else None

# Initialize PyImageJ (no further downloads needed!)
progress.value = 87.5
status_label.value = 'Initializing PyImageJ...'
try:
    # Check if ImageJ is already initialized
    ij.getVersion()
    print(f"PyImageJ already initialized with ImageJ {ij.getVersion()}")
except AttributeError:
    # ij variable is None, safe to initialize
    print(f"Initializing PyImageJ in {ui_mode} mode...")
    ij = imagej.init('./Fiji', mode=ui_mode)
    print(f"PyImageJ initialized with ImageJ {ij.getVersion()}")
except Exception as e:
    # ij exists but might be in bad state, reinitialize
    print(f"Reinitializing PyImageJ (previous state: {e})")
    ij = imagej.init('./Fiji', mode=ui_mode)
    print(f"PyImageJ initialized with ImageJ {ij.getVersion()}")

if ij is not None:
    progress.value = 97
    print("✅ PyImageJ is ready to use! The 'ij' variable contains your ImageJ instance.")
    print(f"UI Mode: {ui_mode}")
    if ui_mode == 'interactive':
        print("💡 Interactive mode enables RoiManager and other GUI features")
    else:
        print("💡 Headless mode - use ij.py.show() for image display")

# 🔍 Utilities 

In [ ]:
#@title ✅ Verify Setup { display-mode: "form" }
#@markdown This double-checks that PyImageJ was setup correctly

status_label.value = 'Verifying PyImageJ functionality...'

# Quick verification that PyImageJ is working
print("=== Setup Verification ===")
print(f"✅ ImageJ version: {ij.getVersion()}")
print(f"✅ Legacy mode: {'Active' if ij.legacy.isActive() else 'Inactive'}")
print(f"✅ UI mode: {ui_mode}")

# Simple test: load and display an image
test_dataset = ij.io().open('https://imagej.net/images/blobs.gif')
print(f"✅ Image loaded: {ij.py.from_java(test_dataset).shape}")
print("🎉 PyImageJ is ready!")
del test_dataset

progress.value += 1

In [ ]:
#@title 🛠️ Troubleshooting { display-mode: "form", run: "auto" }
#@markdown Use this cell if you need to see any hidden output from a failing configuration step.

status_label.value = 'Setting up troubleshooter...'

failing_step = "Select problematic step" #@param ["Select problematic step", "Download PyImageJ", "Set up Fiji Bundle"]
try:
    if failing_step == "Download PyImageJ":
        print("--- Download PyImageJ Output ---")
        print(clone_output.show())
    elif failing_step == "Set up Fiji Bundle":
        print("--- Set up Fiji Bundle Output ---")
        print(bundle_setup.show())
except Exception as e:
    print("No captured output for this stage.", e)

progress.value += 1

In [ ]:
#@title 📋 Copy AI Instructions { display-mode: "form" }
#@markdown If you want to use our configuration with other LLMs (ChatGPT, Claude, etc.), 
#@markdown you can copy the instruction text with the button created by this cell.
#@markdown
#@markdown 💡 Make sure the [📦 Download LLM Rules](#scrollTo=_Download_LLM_Rules) and [🤖 Customize Gemini](#scrollTo=_Customize_Gemini) cells were run first!

status_label.value = 'Setting up LLM copy button...'

from IPython.display import HTML, display
import base64

def make_copy_button(text: str, label: str, icon: str) -> str:
    text_b64 = base64.b64encode(text.encode("utf-8")).decode("ascii")
    return f"""
<button onclick="
    const encodedText = '{text_b64}';
    const byteChars = atob(encodedText);
    const byteNumbers = new Array(byteChars.length);
    for (let i = 0; i < byteChars.length; i++) {{
        byteNumbers[i] = byteChars.charCodeAt(i);
    }}
    const byteArray = new Uint8Array(byteNumbers);
    const decodedText = new TextDecoder('utf-8').decode(byteArray);

    navigator.clipboard.writeText(decodedText).then(() => {{
        this.innerHTML = '✅ Copied to clipboard!';
        setTimeout(() => {{ this.innerHTML = '{icon} {label}'; }}, 2000);
    }}).catch(() => {{
        this.innerHTML = '❌ Copy failed - please copy manually from output above';
        setTimeout(() => {{ this.innerHTML = '{icon} {label}'; }}, 3000);
    }})
">{icon} {label}</button>
"""

copy_button_html = make_copy_button(gemini_text, "Copy LLM Instructions", "🤖")

button_bar_html = f'''
<div style="display: flex; gap: 15px; align-items: center; margin: 10px 0;">
    {copy_button_html}
</div>
'''
display(HTML(button_bar_html))

progress.value += 1

if progress.value < 99:
    status_label.value = '❌ Errors During Setup'
else:
    progress.value = 100
    status_label.value = '✅ Setup complete!'
    # Hide the progress bar, keep only the status label
    progress.layout.visibility = 'hidden'

# 🚀 Start Your ImageJ Journey

Congratulations! You've successfully set up Fiji and PyImageJ, with a custom AI assistant.

## ▶️ **Click the Gemini button** <font color="blue" size="6">(✦)</font> below and start a conversation!

💡You can also just start coding in the following empty cell!

💡 **The `ij` variable is your gateway to ImageJ** - use it to access all ImageJ functionality (e.g., `ij.io().open()`)

Some conversation starters:
* "I work in `[your field]` and am a `[beginning/intermediate/advanced]` user with `[Python/ImageJ]` - build a training notebook tailored to my skills"
* "What can I do with PyImageJ?"
* "How do I load my own image?"
* "Show me how to display an image"
* "Walk me through a basic segmentation workflow"